# 作业一：实现HMM中文分词和BPE英文分词
姓名：薛翔元
学号：521030910387

## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

导入HMM参数，初始化所需的起始概率矩阵，转移概率矩阵，发射概率矩阵

In [226]:
import pickle
import numpy as np

In [227]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的ord作为行key
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子

In [228]:
# TODO: 将input_sentence中的xxx替换为你的姓名（1分）
input_sentence = "薛翔元是一名优秀的学生"

实现viterbi算法，并以此进行中文分词

> 根据算法描述，概率递推公式如下
> 
> $p(h_k) = \left\{ \begin{aligned}
& \pi(h_k) b(h_k,v_k) & k=1 \\
& \max_{h_{k-1}} p(h_{k-1}) a(h_{k-1},h_k) b(h_k,v_k) & 2\leq k\leq n
\end{aligned} \right.$
>
> 其中，$h_k$ 为第 $k$ 个隐状态，$v_k$ 为第 $k$ 个观测状态，$\pi(h_i)$ 表示 $h_i$ 的起始概率，$a(h_i,h_j)$ 表示从 $h_i$ 到 $h_j$ 的转移概率，$b(h_i,v_j)$ 表示从 $h_i$ 到 $v_j$ 的发射概率
>
> 对于最优路径，只需在递推过程中记录下每个 $h_k$ 对应的最优前驱 $h_{k-1}$，回溯即可

In [229]:
def viterbi(sent_orig, start_prob, trans_mat, emission_mat):
    """
    viterbi算法进行中文分词

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)

    #  TODO: 第一个位置的最大概率值计算（1分）
    dp[:, 0] = start_prob * emission_mat[:, sent_ord[0]]

    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）（2分）
    for i in range(1, len(sent_ord)):
        prob = dp[:, i - 1].reshape(-1, 1) * trans_mat * emission_mat[:, sent_ord[i]].reshape(1, -1)
        dp[:, i], path[:, i] = np.max(prob, axis=0), np.argmax(prob, axis=0)

    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]

    #  TODO: 计算labels每个位置上的值（填充labels矩阵）（1分）
    labels[-1] = np.argmax(dp[:, -1])
    for i in range(len(sent_ord) - 2, -1, -1):
        labels[i] = path[labels[i + 1], i + 1]

    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str

In [230]:
print("viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

viterbi算法分词结果： 薛翔/元/是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值

> 前向概率递推公式如下
>
> $\alpha(h_k) = \left\{ \begin{aligned}
& \pi(h_k) b(h_k,v_k) & k=1 \\
& \sum_{h_{k-1}} \alpha(h_{k-1}) a(h_{k-1},h_k) b(h_k,v_k) & 2\leq k\leq n
\end{aligned} \right.$
>
> 最终整个句子的概率值即为 $p(\boldsymbol{v}) = \sum\limits_{h_n} \alpha(h_n)$

In [231]:
def compute_prob_by_forward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    前向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算（1分）
    dp[:, 0] = start_prob * emission_mat[:, sent_ord[0]]

    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(1, len(sent_ord)):
        prob = dp[:, i - 1].reshape(-1, 1) * trans_mat * emission_mat[:, sent_ord[i]].reshape(1, -1)
        dp[:, i] = np.sum(prob, axis=0)

    return sum([dp[i][len(sent_ord) - 1] for i in range(2)])

实现后向算法，计算该句子的概率值

> 后向概率递推公式如下
>
> $\beta(h_k) = \left\{ \begin{aligned}
& 1 & k=n \\
& \sum_{h_{k+1}} \beta(h_{k+1}) a(h_k,h_{k+1}) b(h_{k+1},v_{k+1}) & 1\leq k\leq n-1
\end{aligned} \right.$
>
> 最终整个句子的概率值即为 $p(\boldsymbol{v}) = \sum\limits_{h_1} \beta(h_1) \pi(h_1) b(h_1,v_1)$

In [232]:
def compute_prob_by_backward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    后向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化（1分）
    dp[:, -1] = 1

    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(len(sent_ord) - 2, -1, -1):
        prob = dp[:, i + 1].reshape(1, -1) * trans_mat * emission_mat[:, sent_ord[i + 1]].reshape(1, -1)
        dp[:, i] = np.sum(prob, axis=1)

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [233]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 4.510582343772879e-34
后向算法概率： 4.510582343772878e-34


（TODO：实验总结）

> 在分词部分中，可以看到 Viterbi 算法得到的分词结果基本正确，只有人名未能正确分词，这可能是因为人名在训练语料中频次较低，难以精确得到其发射概率
>
> 在概率计算部分中，可以看到前向算法和后向算法得到的句子概率相等，这在一定程度上验证了算法的正确性
>
> 基于 HMM 的中文分词算法利用动态规划的方法，可以在 $O(n\times d)$ 的时间复杂度下进行高效推理，具有较高的准确率

## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有7处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [234]:
import re

In [235]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")


def white_space_tokenize(corpus):
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到 [["i", "am", "happy"], ["i", "have", "N", "apples"]]

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(filter(lambda tkn: len(tkn) > 0, _splitor_pattern.split(_digit_pattern.sub("N", stc.lower())))) for stc in corpus]

    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典

> 首先依次遍历语料中的每个单词，添加空格和 `</w>` 后存入列表，然后利用 `collections` 模块中的 `Counter` 函数统计频次，得到初始状态词典

In [236]:
def build_bpe_vocab(corpus):
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
     }

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()

    # TODO: 完成函数体（1分）
    from collections import Counter
    split_corpus = [" ".join(list(word)) + " </w>" for stc in tokenized_corpus for word in stc]
    bpe_vocab = dict(Counter(split_corpus))

    return bpe_vocab

编写所需的其他函数

> 遍历每个单词分词状态，利用 `zip` 函数将相邻的 unigram 合并，统计 bigram 频次得到词典

In [237]:
def get_bigram_freq(bpe_vocab):
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：
    输入 bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple(str, str), int] - "bigram->频数"的词典
    """

    bigram_freq = dict()

    # TODO: 完成函数体（1分）
    for word, freq in bpe_vocab.items():
        unigrams = word.split(" ")
        for pair in zip(unigrams[:-1], unigrams[1:]):
            bigram_freq[pair] = bigram_freq.get(pair, 0) + freq

    return bigram_freq

> 遍历每个单词分词状态，利用正则表达式匹配指定的 bigram，并将其替换为合并后的 unigram

In [238]:
def refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab):
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：
    输入 bigram=('i', '</w>')，old_bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

    Args:
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """

    new_bpe_vocab = dict()

    # TODO: 完成函数体（1分）
    pattern = re.compile(r"(^|\s)(" + bigram[0] + r")\s(" + bigram[1] + r")($|\s)")
    for word, freq in old_bpe_vocab.items():
        result = pattern.sub(r"\1" + bigram[0] + bigram[1] + r"\4", word)
        new_bpe_vocab[result] = new_bpe_vocab.get(result, 0) + freq

    return new_bpe_vocab

> 首先构建字典统计频次，然后利用 `sorted` 函数以分词长度为键值进行排序，其中 `</w>` 被替换为 `#`，视为单个字符

In [239]:
def get_bpe_tokens(bpe_vocab):
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：
    输入 bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    得到
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
     ]

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """

    # TODO: 完成函数体（2分）
    bpe_tokens = dict()
    for word, freq in bpe_vocab.items():
        tokens = word.split(" ")
        for token in tokens:
            bpe_tokens[token] = bpe_tokens.get(token, 0) + freq
    bpe_tokens = sorted(bpe_tokens.items(), key=lambda x: len(x[0].replace("</w>", "#")), reverse=True)

    return bpe_tokens

> 顺序遍历 BPE 分词列表，利用正则表达式匹配指定分词，然后递归地对左右两个子串进行分词，最终合并分词结果

In [240]:
def print_bpe_tokenize(word, bpe_tokens):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，
    或者剩余部分无法匹配（该部分整体由"<unknown>"代替）
    
    例1：
    输入 word="supermarket", bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
    最终打印 "su per <unknown>"

    例2：
    输入 word="shanghai", bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
    最终打印 "sh an g hai </w>"

    Args:
        word: str - 待分词的单词str
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """

    # TODO: 请尝试使用递归函数定义该分词过程（2分）
    def bpe_tokenize(sub_word):
        if len(sub_word) == 0:
            return ""
        for token, _ in bpe_tokens:
            result = re.match(r"(.*)(" + token + r")(.*)", sub_word)
            if result:
                return " ".join([bpe_tokenize(result.group(1)), token, bpe_tokenize(result.group(3))]).strip()
        return "<unknown>"

    res = bpe_tokenize(word + "</w>")
    print(res)

开始读取数据集并训练BPE分词器

In [241]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


> 在每次迭代中，首先利用 `get_bigram_freq` 函数统计 bigram 频次，然后取出频次最高的 bigram，利用 `refresh_bpe_vocab_by_merging_bigram` 函数将其合并为 unigram 并更新词典

In [242]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑（2分）
    bigram_freq = get_bigram_freq(training_bpe_vocab)
    bigram = max(bigram_freq, key=lambda x: bigram_freq[x])
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(bigram, training_bpe_vocab)

training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)

测试BPE分词器的分词效果

In [243]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n a tur all an g u ag e pro c es s ing</w>


（TODO：实验总结）

> 从实验结果中可以看到，BPE 算法基本正确地分割了英文子词，从而有效地降低了词表大小
>
> 假定 BPE 分词列表长度为 $d$，则 BPE 算法在长度为 $n$ 的单词上分词的时间复杂度为 $O(n\times d)$，由于 $n$ 通常较小，BPE 算法具有较快的推理速度
>
> 在实现 BPE 算法时，充分利用语言特性和正则表达式可以大幅简化代码，并提高程序的计算效率